The following is an assignment from a text mining class. Sources used is tweets from Twitter and charting data from billboard.com. Summary and finding are provided at bottom of notebook.

### Import packages

In [2]:
# import modules
import pandas as pd 
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_colwidth', 150) 

### Read in text from Source 1: Website

In [3]:
# Getting popular music songs and artists storing text in object
page = requests.get('https://www.billboard.com/charts/billboard-200')
 
soup = BeautifulSoup(page.text, "lxml")

In [4]:
#print(type(soup))
#print(soup)

In [5]:
# Storing the name of the rank, song, and artist and company value in 3 objects

rank = soup.find_all("span",{"class":"chart-element__rank__number"} )
songinfo = soup.find_all("span",{"class":"chart-element__information__song"} )
artistinfo = soup.find_all("span",{"class":"chart-element__information__artist"} )

#print(rank)
#print(song)
#print(artist)
 

### For each source create a data frame containing at least one column of text, two descriptive fields (i.e. source, date, likes, etc), and one category/classification (label)

In [6]:
import time

rankinfo = {}

#Tag each with metadata about which age it came from and when we pulled it
#This also enforces the order in which the data elements are stored
for info in rank:
    rankinfo.setdefault(info.get_text(strip=True), [])
    rankinfo[info.get_text(strip=True)].append(soup.head.title)
    rankinfo[info.get_text(strip=True)].append(time.strftime("%m/%d/%Y"))
    rankinfo[info.get_text(strip=True)].append("Music")
   

In [7]:
# print(rankinfo)

In [8]:
#Storing song info in dictionary

song = {}

for s in songinfo:
    song.setdefault(s.get_text(strip=True), []) 
    
# print(song)

In [10]:
#Storing artist info in dictionary

artist = {}

for a in artistinfo:
    artist.setdefault(a.get_text(strip=True), []) 
    
#print(artist)

In [29]:
# Store the text in three dataframes and combine them into one table

infodf = pd.DataFrame.from_dict(rankinfo,orient="index") 
infodf.reset_index(level=[0], inplace=True)

infodf1 = pd.DataFrame.from_dict(song,orient="index") 
infodf1.reset_index(level=[0], inplace=True) 

infodf2 = pd.DataFrame.from_dict(artist,orient="index") 
infodf2.reset_index(level=[0], inplace=True) 

infodf.columns = ['Ranking', 'Source', 'Date', 'Category']
infodf1.columns = ['Song']
infodf2.columns = ['Artist']

result = pd.concat([infodf, infodf1, infodf2], axis=1)


### Print the Top 5 rows of each data frame

In [30]:
result[0:5]

,Ranking,Source,Date,Category,Song,Artist
0,1,[Billboard 200 Chart | Billboard],09/05/2021,Music,Sour,Olivia Rodrigo
1,2,[Billboard 200 Chart | Billboard],09/05/2021,Music,Trip At Knight,Trippie Redd
2,3,[Billboard 200 Chart | Billboard],09/05/2021,Music,SoulFly,Rod Wave
3,4,[Billboard 200 Chart | Billboard],09/05/2021,Music,Planet Her,Doja Cat
4,5,[Billboard 200 Chart | Billboard],09/05/2021,Music,Solar Power,Lorde


### Read in text from source 2: Twitter

In [31]:
import tweepy
from tweepy import OAuthHandler

#Keys for Twitter
consumer_key =  'mm5L64OCN8CUqdOaibh0PKrHu'
consumer_secret = 'b8QEqj5Lf9auJWEEeEyRf0YIOpqNI73svgTANn0HlzpRm4XUWO'
access_token = '371646061-FZfHfOJQdarWBSMgEZihymDqHubkF1JlCd7H0j3g'
access_secret = '4LW56xEIFI5XF6ODqBoL4MWEHXdee4l0sXHK3XPIdpTk3'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 

api = tweepy.API(auth)

In [33]:
# Converts a given list of tweets into a Pandas DataFrame.
# Code taken from class


def toDataFrame(tweets):

    DataSet = pd.DataFrame()

    #DataSet['tweetID'] = [tweet.id for tweet in tweets]
    DataSet['tweetText'] = [tweet.text for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    #DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['tweetCreated'] = [tweet.created_at for tweet in tweets]
    #DataSet['userID'] = [tweet.user.id for tweet in tweets]
    #DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userName'] = [tweet.user.name for tweet in tweets]
    #DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    #DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    #DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]

    return DataSet

In [34]:
# Create a list of tweets
topicResults = []

#Get the first 100 tweets that mention "@Olivia_Rodrigo" and put them in the topicResults list
#Using Olivia Rodrigo since she currenlty has the number 1 ranking song at the moment
for tweet in tweepy.Cursor(api.search, q='@Olivia_Rodrigo').items(100):
    topicResults.append(tweet)

### For each source create a data frame containing at least one column of text, two descriptive fields (i.e. source, date, likes, etc), and one category/classification (label)¶

In [35]:
#Pass the topicResults list to create a DataFrame
tweet_df = toDataFrame(topicResults)

#Adding an artist column so we know who we are searching about. in this case Olivia Rodrigo
tweet_df['Artist'] = 'Olivia Rodrigo' 

In [36]:
#Renaming columns
tweet_df.columns = ['Text', 'Retweets', 'Favorites', 'Created Date', 'Username', 'User Description', 'Number of Followers', 'User Location', 'Artist']


### Print the Top 5 rows of each data frame

In [41]:
tweet_df.head(5)

,Text,Retweets,Favorites,Created Date,Username,User Description,Number of Followers,User Location,Artist
0,RT @chartdata: Biggest US album debuts of 2021:\n#1 DONDA @kanyewest 309K\n#2 SOUR @Olivia_Rodrigo 295K\n#3 Fearless (Taylor's Version) @taylo…,172,0,2021-09-06 00:50:22,Gordy,,32,"Fort Worth, TX",Olivia Rodrigo
1,RT @chartwomen: Most streamed female artists on Spotify in 2021 so far: \n\n#1 @TaylorSwift13 5.4B\n#2 @ArianaGrande 4.5B\n#3 @Olivia_Rodrigo 4…,139,0,2021-09-06 00:49:54,Constança🧣,don't blame me and wildest dreams stan🖤🤍\n fan account || 15 || Portugal🇵🇹 ||\ntn×1,1832,"Lisboa, Portugal",Olivia Rodrigo
2,@chartdata @kanyewest @Olivia_Rodrigo @taylorswift13 @JColeNC @MorganWallen @billieeilish Hw abt Pure Album Sales,0,0,2021-09-06 00:49:06,gbzgt,Filipino,213,,Olivia Rodrigo
3,"RT @PopCrave: ‘SOUR’ by @Olivia_Rodrigo is #3 on this week’s Billboard 200 after moving a further 74,000 units.\n\nIt previously spent five w…",41,0,2021-09-06 00:48:40,jen signed up for this,"safe space dms always open🫂bisexual🐝💓21 yo swedish wicca girl,engaged💍aries♈️ BLM & LBTQ+ LIFE’S MATTER👊🏽👊🏾👊🏿🏳️‍🌈🏳️‍⚧️tmylm tour june 2nd 2018🧸",2092,She/they,Olivia Rodrigo
4,"RT @chartdata: Billboard 200: #3(-2) @Olivia_Rodrigo, SOUR 74,000 (1,650,000 units since release). *peak: #1 for 5 weeks*",81,0,2021-09-06 00:47:43,AE AE AE,เป็นอากามี่บลิ้งค์ค่ะ,15,,Olivia Rodrigo


### Q1. Write a short description of data you have captured and how they might be used. Make sure your answer is no longer than two paragraphs, and should at minimum answer these questions:



#### What topic are the data related to and why is it relevant to your company?
#### What type of information can we get from the text?
#### What non-text fields did you include and why?

The data I decided to look at is current popular music, songs, and artists. I believe this data can be used from a company marketing standpoint. In regards to building brand partnerships with popular artists that can increase the companies brand awareness awareness among the artists fan demographic for our company products where are target consumers overlaps. These can be social media influencing, playing the song more in store or advertising, and celebrity endorsements.

From the text we are able to see the current popular artists and their music. We are also able to see current sentiment towards these artists through tweet texts and get a glimpse into their fanbase to see if there are overlaps with our target consumer. I included Favorites, retweets, follower counts, rankings and dates. These can give a glimpse into the artist popularity and current trends in the pop music space.



### Q2. Write a short description comparing the context surrounding each of the two data sources. Make sure your answer is no longer than three paragraphs, and should at minimum answer these questions:

#### How different is the context in which the data were generated (e.g. news vs. social media)?
#### How technically different was the collection and manipulation of these two data sources?
#### How do the contextual and technical differences influence the data’s relevance or usefulness for your company?

The news context is that it is showing a ranking based on song streams and sales that is quantifiable facts. Social media is able to show more opinions of the artist and is changing as time goes on. For example, through the social media data I was able to find out that Olivia Rodrigo had a recent scandal involving her music which may cause our company to look for other artists. The news just shows that she has the most popular song.

The technical collection was a bit tougher sifting thorough the html on the website to find the relevant information to pull in. Thev results from the news collection informed what the collections of social media would be scraped.

The datas relevance is informed first by the news collection. This is useful in finding a very broad scope of current popularity among current artists and songs. From here we are then able to focus more specifically on these popular artists indiviually to find things like fan sentiment, fan demographic, popularity, scandals, and ultimately to see if this popular artist may be a good candidate to further look into as a partner in a marketing campaign,